# Aprendendo mais conceitos construindo uma FNN com Pytorch
- A intenção desse Notebook é aprendermos mais alguns conceitos muito importantes já implementando uma rede neural
- Os conceitos são:
     - Carregar um dataset usando `torchvision.datasets`
     - Entender o `torch.utils.data.Dataset`
     - Entender o `torchvision.transform`
     - Aprender a usar um `Dataloader`
     - Criar um modelo estendendo a classe `nn.Module`
     - Treinar a rede em uma GPU 

In [ ]:
import torchvision
import matplotlib.pyplot as plt
import torchvision.transforms as transforms
import torch
import torch.nn as nn

## Utilizando a base CIFAR10
- [Dataset muito famoso](https://www.cs.toronto.edu/~kriz/cifar.html) que contém 60k imagens coloridas 32 x 32 com 10 classes de 6k imagens por classe
- O conjunto de teste possui 10k imagens
- Vamos carregar ela utilizando  `torchvision.datasets` 
    - `torchvision` é um pacote do PyTorch que contém datasets, modelos e transformações famosas da área de deep learning
    - Vamos usar bastante esse pacote nesse módulo
    - [Documentação](https://pytorch.org/vision/stable/index.html)   
- Vamos começar usando `torchvision.datasets.CIFAR10`
    - [Documentação](https://pytorch.org/vision/stable/generated/torchvision.datasets.CIFAR10.html#torchvision.datasets.CIFAR10)
- Para facilitar o tempo de execução, vamos carregar apenas o conjunto de teste (10k imagens)

In [ ]:
cifar_dataset = torchvision.datasets.CIFAR10(root="/home/patcha/datasets", 
                                             train=False, 
                                             download=True)

- O Pytorch tem uma classe especial chamada `torch.utils.data.Dataset`
    - [Documentação](https://pytorch.org/docs/stable/data.html?highlight=dataset#torch.utils.data.Dataset)
- Ela é uma maneira de preparar e carregar dados dentro do framework
- Ela implementa a `__getitem__`

In [ ]:
cifar_dataset

In [ ]:
len(cifar_dataset)

- O Pytorch utiliza uma biblioteca para manipulação de imagens chamada [Pillow](https://pillow.readthedocs.io/en/stable/reference/Image.html)
    - Por enquanto, apenas aceite que ela carrega imagens assim como o OpenCV

In [ ]:
cifar_dataset[0]

In [ ]:
img, label = cifar_dataset[2]
plt.imshow(img)

- Agora vamos criar um `Dataloader`, que nada mais é do que um classe que vai criar um `Generator` usando um `Dataset`
    - [Documentação](https://pytorch.org/docs/stable/data.html)
- **Nota**: se você não sabe nada sobre generators, eu sugiro a [leitura desse post](https://realpython.com/introduction-to-python-generators/)

In [ ]:
batch_size = 10
train_loader = torch.utils.data.DataLoader(dataset=cifar_dataset, 
                                           batch_size=batch_size, 
                                           shuffle=True)

In [ ]:
train_loader

- Podemos acessar dados de um dataloader dentro de um loop
- Porém, tem um problema: **dataloaders não aceitam PIL como entrada**
    - Se rodarmos o código abaixo vamos obter um erro

In [ ]:
for img, labels in train_loader:
    plt.imshow(img)
    break

- Precisamos uma maneira de converter os dados para tensores ou `np.ndarray`
- Felizmente, a `torchvision` possui um submódulo para aplicar transformações nos dados
    - [`torchvision.transforms`](https://pytorch.org/vision/stable/transforms.html)
- É muito útil para aplicarmos data augmentation, mas isso é tema para próxima aula
- Aqui, vamos usar apenas o `transforms.ToTensor()`, que converte um PIL ou `np.ndarray` para um tensor
- Na verdade, podemos fazer isso dentro da chamada do `datasets` do Pytorch:

In [ ]:
cifar_dataset = torchvision.datasets.CIFAR10(root="/home/patcha/datasets", 
                                             train=False, 
                                             download=True,
                                            transform=transforms.ToTensor())

- Agora, os dados das imagens serão tensores:

In [ ]:
cifar_dataset[0]

- E podemos recriar nosso `Dataloader:

In [ ]:
batch_size = 10
train_loader = torch.utils.data.DataLoader(dataset=cifar_dataset, 
                                           batch_size=batch_size, 
                                           shuffle=True)

- E podemos acessar também:
    - Porém, sempre é retornado um batch de imagens!

In [ ]:
for batch_img, batch_labels in train_loader:
    print(batch_img.shape)
    break

- Essa é uma boa hora para entender esse tensor
- No Pytorch, quando trabalhamos com imagens, sempre teremos:
    - Dimensão 0: batch size
    - Dimensão 1: canais da imagem
    - Dimensão 2: width
    - Dimensão 3: height           

## Criando uma rede neural do tipo Feedforward
- Agora vamos criar nossa rede neural
- A maneira padrão de criar redes neurais dentro do Pytorch é estendendo a classe `nn.Module`
- Você pode imaginar a criação de redes como conjunto de blocos que vamos compondo
    - Como um LEGO

In [ ]:
class FNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_labels):
        super(FNN, self).__init__()
        
        # Aqui vamos definir a arquitetura da rede
        self.fc1 = nn.Linear(input_size, hidden_size) 
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, num_labels)  
        self.soft = nn.Softmax(num_labels)

    def forward(self, x, apply_soft=False):
        """
        Esse método precisa ser criado para fazermo o forward pass da rede
        """
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        if apply_soft:
            out = self.soft(out)
        return out

- Agora podemos instanciar o nosso modelo:

In [ ]:
input_size = 32*32*3
model = FNN(input_size, 20, 10)
model

- Agora podemos determinar nossa função de custo e otimizador
- Para esse notebook vamos usar a Entropia cruzada e o otimizador Adam (uma variação do gradiente descendente)

In [ ]:
loss_func = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)  

- Agora vamos fazer nosso loop de treinamento
- Agora, vamos mandar nosso modelo para GPU se ela estiver disponível

In [ ]:
num_epochs = 10
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Movendo o modelo para o device alvo
model.to("cuda")

for epoch in range(num_epochs):
    for k, (batch_images, batch_labels) in enumerate(train_loader):  
        
        # Aplicando um flatten na imagem e movendo ela para o device alvo
        batch_images = batch_images.reshape(-1, 32*32*3).to(device)
        batch_labels = batch_labels.to(device)
        
        # Fazendo a forward pass
        # observe que o modelo é agnóstico ao batch size
        outputs = model(batch_images)
        loss = loss_func(outputs, batch_labels)
        
        # Fazendo a otimização
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()        
    
    print (f"- Epoch [{epoch+1}/{num_epochs}] | Loss: {loss.item():.4f}")                  

### Fazendo inferência
- Como estamos usando apenas um pedaço do dataset, vamos fazer um exemplo de inferência com um pedaço do conjunto de treino
    - O ideal era ter validação e teste
- A inferência é basicamente igual a do notebook anterior

In [ ]:
with torch.no_grad():
    correct, total = 0, 0
    for images, labels in train_loader:
        images = images.reshape(-1, 32*32*3).to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print(f"Accuracy: {100 * correct / total}%")

- Salvando o modelo:

In [ ]:
torch.save(model.state_dict(), 'model.pth')

___
# Exercício
- Estude o código desse notebook e faça as seguinte adaptações:
    - Carregue os dados da CIFAR10 inteiro (teste e treino)
    - Divida os conjuntos em treino, teste e validação
    - Inclua uma avaliação das métricas para o conjunto de validação e acompanhe as curvas de aprendizado
    - Faça a inferência no conjunto de teste e compare com a validação
___